# Hugging Face

## 0. Environment Settings 环境配置

- 在conda环境中安装基础python依赖：
    ```bash
    conda create -n transformer
    conda activate transformer
    pip install transformer datasets evaluate peft accelerate gradio optimum sentencepiece jupyterlab scikit-learn pandas matplotlib tensorboard nltk rouge
    ```

- 网络问题更改本地电脑hosts：*本条操作待验证*
  - hosts储存路径为: *C:\Windows\System32\drivers\etc\hosts*
  - 添加如下内容：

## 1. Pipeline 工具

### 1.1 Pipeline 概念


[Pipeline Official Doc](https://huggingface.co/transformers/main_classes/pipelines.html#transformers.pipeline)

- pipeline是一个封装好的工具，可以通过直接的声明进行调用，完成一些预先设定好的NLP任务

- pipeline的大致syntax为（较常用的一些opt参数）：`pipeline(task="", model="", tokenizer="", device=)`

![20230716105734](https://michael-1313341240.cos.ap-shanghai.myqcloud.com/20230716105734.png)


### 1.2 Pipeline支持的Task


- task主要可以包括的任务有：
  - 文本分类（包含情感分析）: `task = "text-classification"` `task = "zero-shot-classification"`
  - Token 分类（包含命名实体识别 NER）: `task = "token-classification"`
  - 人机对话: `task = "conversational"`
  - 文档问答: `task = "document-question-answering"`
  - 问题回答: `task = "question-answering"`
  - 表格回答: `task = "table-question-answering"`
  - 填空: `task = "fill-mask"`
  - 生成摘要: `task = "summarization"`
  - Text-to-text 文本生成: `task = "text2text-generation"`
  - 机器翻译: `task = "translation"`
  - 跨模态任务: `task = "image-to-text"` `task = "visual-question-answering"`


*可以通过下列命令查看 **pipeline 支持的所有任务***

In [1]:
from transformers.pipelines import SUPPORTED_TASKS
print(*SUPPORTED_TASKS.items(),sep="\n")

('audio-classification', {'impl': <class 'transformers.pipelines.audio_classification.AudioClassificationPipeline'>, 'tf': (), 'pt': (<class 'transformers.models.auto.modeling_auto.AutoModelForAudioClassification'>,), 'default': {'model': {'pt': ('superb/wav2vec2-base-superb-ks', '372e048')}}, 'type': 'audio'})
('automatic-speech-recognition', {'impl': <class 'transformers.pipelines.automatic_speech_recognition.AutomaticSpeechRecognitionPipeline'>, 'tf': (), 'pt': (<class 'transformers.models.auto.modeling_auto.AutoModelForCTC'>, <class 'transformers.models.auto.modeling_auto.AutoModelForSpeechSeq2Seq'>), 'default': {'model': {'pt': ('facebook/wav2vec2-base-960h', '55bb623')}}, 'type': 'multimodal'})
('feature-extraction', {'impl': <class 'transformers.pipelines.feature_extraction.FeatureExtractionPipeline'>, 'tf': (), 'pt': (<class 'transformers.models.auto.modeling_auto.AutoModel'>,), 'default': {'model': {'pt': ('distilbert-base-cased', '935ac13'), 'tf': ('distilbert-base-cased', '9

### 1.3 Pipeline的创建与使用（例）

- 在HuggingFace的[Model](https://huggingface.co/models)分区可以寻找想要用的模型
- 点击进入模型详细页面后，可以看到具体描述和demo交互

![20230716120431](https://michael-1313341240.cos.ap-shanghai.myqcloud.com/20230716120431.png)

In [14]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer,pipeline
# model是指定这个task要使用的nlp模型
model = AutoModelForSequenceClassification.from_pretrained('uer/roberta-base-finetuned-chinanews-chinese')
# tokenizer是指定task的分词器，其具体含义会在后面进行学习
tokenizer = AutoTokenizer.from_pretrained('uer/roberta-base-finetuned-chinanews-chinese')
# 通过pipeline(...)就会生成这样一个封装好的工具，这里命名为pipe
## 此处的task为sentiment analysis，即情感分析
pipe = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)
# 调用pipe，配合相应的输入，就可以得到nlp任务的相应输出
pipe("暴雪被微软收购了！")

[{'label': 'financial news', 'score': 0.5913024544715881}]

**使用GPU加速计算**

通过`pipeline(, ... , device = 0)`显式加载第0张GPU进行加速计算

In [29]:
import torch
import time
times = []

# GPU inference
pipe_gpu = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer,device=0)
for i in range(1000):
    torch.cuda.synchronize()
    start = time.time()
    pipe_gpu("暴雪被微软收购了！")
    torch.cuda.synchronize()
    end = time.time()
    times.append(end - start)
print(f"GPU:{sum(times) / 1000}")

# CPU inference
for i in range(1000):
    torch.cuda.synchronize()
    start = time.time()
    pipe("暴雪被微软收购了！")
    torch.cuda.synchronize()
    end = time.time()
    times.append(end - start)
print(f"CPU:{sum(times) / 1000}")

GPU:0.005329703807830811
CPU:0.01029831838607788


### 1.4 Pipeline的背后实现

**1. 初始化Tokenizer**

In [19]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
tokenizer = AutoTokenizer.from_pretrained\
    ("uer/roberta-base-finetuned-dianping-chinese")

**2. 初始化Model**

In [15]:
model = AutoModelForSequenceClassification.from_pretrained\
    ("uer/roberta-base-finetuned-dianping-chinese")

In [28]:
# 查看模型结构
print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

**3. 数据输入&通过`tokenizer`预处理**

In [16]:
input_txt = "我很喜欢你！"
input = tokenizer(input_txt, return_tensors="pt") # pt即pytorch

In [33]:
# 查看分词后结果
print(input)

{'input_ids': tensor([[ 101, 2769, 2523, 1599, 3614,  872, 8013,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}


**4. 模型预测**

In [38]:
res = model(**input) # 这一步得到的结果是一个未标准化的logits向量
print(f"res:\n{res}") # 输出res，这里列出了一系列中间操作的说明；数值上重点关注logits值
logits = res.logits # 从res中提取logits向量
print(f"logits:\n{logits}") # 输出logits向量

res:
SequenceClassifierOutput(loss=None, logits=tensor([[-1.4593,  1.4171]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
logits:
tensor([[-1.4593,  1.4171]], grad_fn=<AddmmBackward0>)


**5. 输出结果**

In [49]:
logits_softmax = torch.softmax(logits,dim=-1) # 对logits向量进行softmax归一化操作
print(f"logits_softmax:\t{logits_softmax}") # 输出归一化后的logits向量

pred = torch.argmax(logits_softmax).item() # 从归一化后的logits向量中提取最大值的索引 
print(f"pred:\t{pred}") # 输出预测结果，这里的pred是一个数字索引，需要转换为索引对应的标签

result = model.config.id2label.get(pred) # 通过id2label字典，将索引转换为标签
print(f"result:\t{result}") # 输出标签

logits_softmax:	tensor([[0.0533, 0.9467]], grad_fn=<SoftmaxBackward0>)
pred:	1
result:	positive (stars 4 and 5)


## 2. Tokenizer

### 2.1 Tokenizer 概念

**基本NLP数据预处理流程**

1. 分词：将文本数据分词成字、字词等
2. 构建词典：构建词典，filter一些过低、过高的单词等
3. 数据转换：将词典映射成数字序列
4. 数据填充与截断
   

### 2.2 Tokenizer的使用

In [50]:
from transformers import AutoTokenizer
demo = "那些痛的记忆，落在春的泥土里"

**1. 加载Tokenizer**

In [52]:
# 从huggingface的模型库中加载分词器
tokenizer = AutoTokenizer.from_pretrained\
    ("uer/roberta-base-finetuned-dianping-chinese")
print(tokenizer)

BertTokenizerFast(name_or_path='uer/roberta-base-finetuned-dianping-chinese', vocab_size=21128, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)


*也可以将tokenizer保存到本地再从本地调用*

In [ ]:
tokenizer.save_pretrained("./roberta_tokenizer")
tokenizer = AutoTokenizer.from_pretrained("./roberta_tokenizer")
print(tokenizer)

**2. 利用`tokenizer.tokenize()`进行分词**

In [55]:
tokens = tokenizer.tokenize(demo)
print(tokens)

['那', '些', '痛', '的', '记', '忆', '，', '落', '在', '春', '的', '泥', '土', '里']


*对于tokenizer词典，可以通过以下方法进行查看：*

`tokenizer.vocab()`：查看词典

`tokenizer.vocab_size`：查看词典大小

**3\*. 利用`tokenizer.convert_to_ids()`进行索引转换**

目的：token -> id

In [57]:
# 将词序列转换为id序列
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

[6929, 763, 4578, 4638, 6381, 2554, 8024, 5862, 1762, 3217, 4638, 3799, 1759, 7027]


*相反地，也可以通过`tokenizer.convert_ids_to_tokens(ids)`进行逆变换*

In [61]:
# 将id序列转换为token序列
tokens = tokenizer.convert_ids_to_tokens(ids)
print(tokens)

# 将token序列转换为string
str_sen = tokenizer.convert_tokens_to_string(tokens)
print(str_sen)

['那', '些', '痛', '的', '记', '忆', '，', '落', '在', '春', '的', '泥', '土', '里']
那 些 痛 的 记 忆 ， 落 在 春 的 泥 土 里


**3. 利用`tokenizer.encode()`快捷转换**

**作用：token -> id**

In [64]:
# 将字符串转换为id序列，又称之为编码
ids = tokenizer.encode(demo, add_special_tokens=True)
print(ids)

[101, 6929, 763, 4578, 4638, 6381, 2554, 8024, 5862, 1762, 3217, 4638, 3799, 1759, 7027, 102]


注：
1. 若和上面的`tokenizer.convert_to_ids()`同时使用，会发现`tokenizer.encode()`的结果会在前后加上`[CLS]`和`[SEP]`，也就是输出中的[101],[102]. 其本身的目的是为了方便进行模型输入的构建。
2. 若不希望加入`[CLS]`和`[SEP]`，可以通过`add_special_tokens=False`进行设置。

相对应，可以通过`tokenizer.decode()`进行逆变换

**作用：id -> token**

In [65]:
# 将id序列转换为字符串，又称之为解码
str_sen = tokenizer.decode(ids, skip_special_tokens=False)
print(str_sen)

[CLS] 那 些 痛 的 记 忆 ， 落 在 春 的 泥 土 里 [SEP]


**4. 填充与截断**

说明：在进行分词的过程中，会有些句子过长或过短，因此需要进行填充与截断以得到长度适中的数据。

**填充**

利用`tokenizer.encode(..., padding = "max_length", max_length = )` 进行填充

In [71]:
# 填充
ids = tokenizer.encode(demo, padding="max_length", max_length=20)
print(ids)
print(tokenizer.decode(ids, skip_special_tokens=False))

[101, 6929, 763, 4578, 4638, 6381, 2554, 8024, 5862, 1762, 3217, 4638, 3799, 1759, 7027, 102, 0, 0, 0, 0]
[CLS] 那 些 痛 的 记 忆 ， 落 在 春 的 泥 土 里 [SEP] [PAD] [PAD] [PAD] [PAD]


*注：观察上面的输出可以发现最后几个位置为0，如果decode回去会发现这里是`[PAD]`，也就是填充的位置。*

**截断**

利用`tokenizer.encode(..., truncation = True, max_length = )` 进行截断

In [69]:
# 截断
ids = tokenizer.encode(demo, max_length=5, truncation=True)
ids

[101, 6929, 763, 4578, 102]

**5. 利用`tokenizer.encode_plus`完成 Attention Mask**

对于某些模型来说，单纯的填充是不够的，需要另外建立一个attention_mask的bool数组来指示那部分是真实输出，那部分是填充内容。

这一功能在调用`tokenizer.encode_plus`时，会自动完成。

In [73]:
inputs = tokenizer.encode_plus(demo, padding="max_length", max_length=15)
print(inputs)

{'input_ids': [101, 6929, 763, 4578, 4638, 6381, 2554, 8024, 5862, 1762, 3217, 4638, 3799, 1759, 7027, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


**！最后的封装**

上述全部流程最后被完全封装进了`tokenizer`中，因此在实际操作中，可以直接调用`tokenizer`进行数据预处理。